In [99]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Flatten, Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet50 import ResNet50

In [2]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.2)

sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [13]:

def build_densenet121(train_model = True, **kwargs):
    weights = kwargs.get('weights', 'imagenet')
    #num_classes = kwargs.get('num_classes')
    
    inputs = Input(shape=(256, 256, len(kwargs.get('channels')) ))
    dense_filter = tf.keras.layers.Conv2D(3, 3, padding='same')(inputs)

    densenet = DenseNet121(include_top=False, weights=weights)(dense_filter)
    #x = densenet.output
    x = Conv2D(filters=128,kernel_size=(4,4),activation='relu')(densenet) # 8
    x = Conv2D(filters=64,kernel_size=(1,1),activation='relu')(x)
    preds = Conv2D(filters=2, kernel_size=(1,1),activation='softmax')(x) 
    if train_model:
        preds = Flatten()(preds)
    model = Model(inputs=inputs, outputs=preds)

    #model.summary()
    model.summary()
    return model

In [15]:

build_densenet121(**{'channels': ['B4','B3','B2', 'NDWI', 'AVE']})

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 256, 256, 5)]     0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 256, 256, 3)       138       
_________________________________________________________________
densenet121 (Model)          multiple                  7037504   
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 5, 5, 128)         2097280   
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 5, 5, 64)          8256      
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 5, 5, 2)           130       
_________________________________________________________________
flatten_3 (Flatten)          (None, 50)                0   

In [107]:
inputs = Input(shape=(128, 128, 5))
resnet = ResNet50(include_top=False, weights=None, input_tensor=inputs)
resnet.summary()


x = resnet.output
x = Conv2D(filters = 512, kernel_size=(4,4), activation='relu')(x)
preds = Conv2D(filters = 2, kernel_size=(1,1), activation='relu')(x)

#preds = Dense(2, activation='softmax', name='fc1000')(x)

model = Model(inputs=resnet.input, outputs=preds)
model.summary()
#x = Conv2D(filters = 512, kernel_size=(4,4), activation='relu')(resnet)
#preds = Conv2D(filters = 2, kernel_size=(1,1), activation='relu')(x)

#model = Model(inputs=inputs, outputs=resnet)
#model.summary()


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_170 (InputLayer)          [(None, 128, 128, 5) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 5)  0           input_170[0][0]                  
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 64, 64)   15744       conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 64, 64, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________